<h1>Neural Network SMS Text Classifier ML </h1>

In [3]:
# import libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.1


In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-04-12 20:05:54--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::ac43:4695, 2606:4700:20::681a:321, 2606:4700:20::681a:221, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::ac43:4695|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349,84K  --.-KB/s    in 0,02s   

2024-04-12 20:05:54 (21,1 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-04-12 20:05:54--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::681a:221, 2606:4700:20::681a:321, 2606:4700:20::ac43:4695, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::681a:221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab

In [36]:
# Make dataframe of dataset to see data

dataset_train = pd.read_csv(train_file_path, sep='\t')

# rename the second column to 'text'
list_columns = dataset_train.columns.tolist()
list_columns[1] = 'text'
dataset_train.columns = list_columns

dataset_train.head(10)

,ham,text
0,ham,you can never do nothing
1,ham,"now u sound like manky scouse boy steve,like! ..."
2,ham,mum say we wan to go then go... then she can s...
3,ham,never y lei... i v lazy... got wat? dat day ü ...
4,ham,in xam hall boy asked girl tell me the startin...
5,ham,genius what's up. how your brother. pls send h...
6,ham,they finally came to fix the ceiling.
7,spam,urgent! call 09066350750 from your landline. y...
8,ham,now that you have started dont stop. just pray...
9,ham,hello darlin ive finished college now so txt m...


In [37]:
dataset_test = pd.read_csv(test_file_path, sep='\t')

# rename the second column to 'text'
list_columns = dataset_test.columns.tolist()
list_columns[1] = 'text'
dataset_test.columns = list_columns

dataset_test.head(10)

,ham,text
0,ham,"not much, just some textin'. how bout you?"
1,ham,i probably won't eat at all today. i think i'm...
2,ham,don‘t give a flying monkeys wot they think and...
3,ham,who are you seeing?
4,ham,your opinion about me? 1. over 2. jada 3. kusr...
5,ham,yesterday its with me only . now am going home.
6,ham,yes. it's all innocent fun. o:-)
7,ham,a boy was late 2 home. his father: power of fr...
8,ham,is ur changes 2 da report big? cos i've alread...
9,ham,sorry about earlier. putting out fires.are you...
